In [1]:
import networkx as nx
import numpy as np

In [2]:
def graph(fileIn = None):
    """
    input: 
        fileIn: fileName input for graph 
    output: G
    """
    G = {}
    try:
        f = open(fileIn, 'r')
        datas = f.readlines()
        f.close()
    except:
        print('file not exists!!')
        return None
    max_node = 0
    for line in datas:
        if line[0] != '#':
            pair = line.split()
            f, t = int(pair[0]), int(pair[1])
            G[f] = G.get(f, [])
            G[t] = G.get(t, [])
            G[f].append(t)
    return G

In [3]:
# đọc đồ thị dữ liệu từ file
G = graph('web-NotreDame.txt')
len(G.keys())

325729

In [4]:
def pageRank(G, beta = 0.85, iter = 100, teleport_list = None, eps = 1e-8):
    if not teleport_list:
        teleport_list = G.keys()
    N = len(G.keys())
    # Khởi tạo rank ban đầu
    next_rank_list = np.array([1/N for i in range(N)])
    curr_rank_list = np.array(next_rank_list)
    const_rank_list = np.zeros(N)
    for node in teleport_list:
        const_rank_list[node] = (1.0 - beta) / N
    
    # Lặp để cập nhật rank cho các node
    for i in range(iter):
        curr_rank_list, next_rank_list = np.array(next_rank_list), np.array(const_rank_list)
        
        for node in G:
            if G[node]:
                contribution = beta * curr_rank_list[node] / len(G[node])
                for edge in G[node]:
                    next_rank_list[edge] += contribution
        
        leakage_contribution = (1.0 - np.sum(next_rank_list)) / N
        next_rank_list += leakage_contribution
        total_diff = np.sum(np.abs(curr_rank_list - next_rank_list))
        if total_diff < eps * N: #đưa về cùng điều kiện so sánh tol với thuật toán pagerank của thư viện networkx
            break

    return next_rank_list

In [5]:
%time rank_list = pageRank(G, eps=1e-08)
print(np.sum(rank_list))

Wall time: 13 s
1.0000000000000002
0.005524903242845631


In [6]:
rank_list

array([5.52490324e-03, 4.85749916e-04, 2.78690533e-04, ...,
       2.84880796e-06, 2.90259020e-06, 2.04667652e-06])

## Using NetworkX

In [7]:
diG = nx.DiGraph()
for key in G:
    for el in G[key]:
        diG.add_edge(key, el)

In [8]:
diG.number_of_nodes()

325729

### PageRank algorithm

In [9]:
%time pagerank = nx.pagerank(diG, tol=1e-08)

Wall time: 1min 22s


### convert pagerank to numpy array

In [10]:
pageRank_list = []
for page in pagerank:
    pageRank_list.append(pagerank[page])

In [11]:
pageRank_list = np.array(pageRank_list)

In [12]:
pageRank_list

array([5.52490324e-03, 4.85749916e-04, 2.78690533e-04, ...,
       2.84880796e-06, 2.90259020e-06, 2.04667652e-06])

## Sử dụng các độ đo so sánh 2 thuật toán

### root mean squared error

In [14]:
np.sqrt(np.mean(np.power((np.array(pageRank_list) - rank_list), 2)))

4.1214435370337915e-18

### absolute mean error

In [15]:
np.mean(np.abs(np.array(pageRank_list) - rank_list))

4.0434020954854955e-19

In [ ]:
pageRank_list - rank_list

### Hits algorithm

In [16]:
# %time hits = nx.hits(diG, tol=1e-04)

Wall time: 10min 26s


In [25]:
#hits_list = []
#for page in hits[1]:
#    hits_list.append(hits[1][page])

### root mean squared error với pageRank_list

In [29]:
#np.sqrt(np.mean(np.power((np.array(hits_list) - pageRank_list), 2)))

4.7968250026397676e-05

### absolute mean error với pageRank_list

In [28]:
#np.mean(np.abs(np.array(hits_list) - pageRank_list))

5.715781779943196e-06

### root mean squared error với rank_list

In [26]:
#np.sqrt(np.mean(np.power((np.array(hits_list) - rank_list), 2)))

4.796825002639911e-05

### absolute mean error với rank_list

In [27]:
#np.mean(np.abs(np.array(hits_list) - rank_list))

5.715781779943539e-06